In [10]:
import sys

!python3 -m pip install --upgrade Pillow

                                              0.0/2.5 MB ? eta -:--:--
                                              0.0/2.5 MB 1.3 MB/s eta 0:00:02
     --                                       0.1/2.5 MB 1.7 MB/s eta 0:00:02
     ----                                     0.3/2.5 MB 2.1 MB/s eta 0:00:02
     --------                                 0.6/2.5 MB 3.2 MB/s eta 0:00:01
     -------------------                      1.2/2.5 MB 5.6 MB/s eta 0:00:01
     --------------------------------------   2.4/2.5 MB 9.0 MB/s eta 0:00:01
     ---------------------------------------- 2.5/2.5 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: pillow 10.2.0
    Uninstalling pillow-10.2.0:
      Successfully uninstalled pillow-10.2.0



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\lukez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
from PIL import Image
import numpy as np

In [13]:
x=Image.open('images_original/blues/blues00000.png')
print(x.format, x.size, x.mode)
print(np.array(x.getdata()).flatten())





PNG (432, 288) RGBA
[255 255 255 ... 255 255 255]


In [14]:
IMAGE_COMPONENTS=4
def image_distance(im1,im2):
    acc=0
    for i in range(len(im1)):
        pix1=im1[i]
        pix2=im2[i]
        for ii in range(IMAGE_COMPONENTS):
            acc+=(pix1[ii]-pix2[ii])**2
    return acc**(1/2)
def image_number(index):
     if index<10:
        return '0000'+str(index)
     return '000'+str(index)
    
def imageToNumber(genres,genreIndex,index):
    if genres[genreIndex]!='jazz' or index<54:
        return 100*genreIndex+index
    return 100*genreIndex+index-1
    
    
images_filepath='images_original/'
genres=['blues','classical','country','disco','hiphop','metal','pop','reggae','rock','jazz']
NUM_IMAGES_PER_GENRE=100

In [15]:
genreX=[[] for _ in range(len(genres))]
for genreIndex in range(len(genres)):
    for imageIndex in range(NUM_IMAGES_PER_GENRE):
        imageName1=genres[genreIndex]+image_number(imageIndex)
        if genres[genreIndex]!='jazz' or imageIndex!=54:
            genreX[genreIndex].append(np.array(Image.open(images_filepath+genres[genreIndex]+'/'+imageName1+'.png').getdata()).flatten())
   

for i in range(len(genreX)):
    genreX[i]=np.array(genreX[i])

In [16]:
dists=[]
for i in range(len(genres)):
    print(genres[i])
    for ii in range(i+1,len(genres)):
            for iii in range(NUM_IMAGES_PER_GENRE):
                if genres[i]!='jazz' or ii!=54 and not (genres[i]=='jazz' and ii!=99):
                    for iiii in range(NUM_IMAGES_PER_GENRE):
                        if genres[ii]!='jazz' or iiii!=54 and not (genres[ii]=='jazz' and iiii==99):
                            name1=imageToNumber(genres,i,iii)
                            name2=imageToNumber(genres,ii,iiii)
                            ixx=genreX[i][iii].T@genreX[i][iii]
                            iixx=genreX[ii][iiii].T@genreX[ii][iiii]
                            dists.append((name1,name2,(ixx-2*genreX[i][iii].T@genreX[ii][iiii]+iixx)**(1/2)))
            

print(len(dists))
        

        

blues


C:\Users\lukez\AppData\Local\Temp\ipykernel_23252\4060238381.py:13: RuntimeWarning: overflow encountered in scalar subtract
  dists.append((name1,name2,(ixx-2*genreX[i][iii].T@genreX[ii][iiii]+iixx)**(1/2)))
C:\Users\lukez\AppData\Local\Temp\ipykernel_23252\4060238381.py:13: RuntimeWarning: overflow encountered in scalar add
  dists.append((name1,name2,(ixx-2*genreX[i][iii].T@genreX[ii][iiii]+iixx)**(1/2)))
C:\Users\lukez\AppData\Local\Temp\ipykernel_23252\4060238381.py:13: RuntimeWarning: invalid value encountered in power
  dists.append((name1,name2,(ixx-2*genreX[i][iii].T@genreX[ii][iiii]+iixx)**(1/2)))


classical
country
disco
hiphop
metal
pop
reggae
rock
jazz
448200


In [17]:
dists.sort(key=lambda x: x[2])

In [18]:
def union_find(parents,i):
    while(parents[i]!=i):
        i=parents[i]
    return i
def union_merge(size,parents,p1,p2):
    if size[p1]>size[p2]:
        parents[p2]=p1
        size[p1]+=size[p2]
    else:
        parents[p1]=parents[p2]
        size[p2]+=size[p1]

        
def minSpanTree(edges,numNodes,numGroupings):
    parents=[i for i in range(numNodes)]
    size=[1 for _ in range(numNodes)]
    groupings=numNodes
    i=0
    while groupings>numGroupings:
        
        currEdge=edges[i]
        p1=union_find(parents,currEdge[0])
        p2=union_find(parents,currEdge[1])
        if p1!=p2:
            union_merge(size,parents,p1,p2)
            groupings-=1
            
        i+=1
    return parents,size
        

In [19]:
parents,size=minSpanTree(dists,9*100+99,10)
groups={}
for i in range(len(parents)):
    parent=union_find(parents,i)
    if parent in groups:
        groups[parent]+=1
    else:
        groups[parent]=1
print(groups)


{133: 990, 309: 1, 480: 1, 506: 1, 514: 1, 526: 1, 551: 1, 786: 1, 891: 1, 998: 1}


In [9]:
node0=(0,6)
node1=(-1,7)
node2=(0,0)
node3=(0,1)
node4=(8,9)
node5=(10,11)
nodes=[node0,node1,node2,node3,node4,node5]
edges=[]
for i in range(len(nodes)):
  for j in range(i+1,len(nodes)):
    node1x,node1y=nodes[i][0],nodes[i][1]
    node2x,node2y=nodes[j][0],nodes[j][1]
    edges.append((i,j,((node1x-node2x)**2+(node1y-node2y)**2)**(1/2)))

edges.sort(key=lambda x: x[2])
parents,size=minSpanTree(edges,6,3)
print(parents,size)
groups={}
for i in range(len(parents)):
  parent=union_find(parents,i)
  if parent in groups:
    groups[parent].append(i)
  else:
    groups[parent]=[i]
for i in groups:
  print("cluster composed of ",groups[i])

[1, 1, 3, 3, 5, 5] [1, 2, 1, 2, 1, 2]
cluster composed of  [0, 1]
cluster composed of  [2, 3]
cluster composed of  [4, 5]
